# Group Analysis

Desired output figure:
- time x classification accuracy
- Within, Pre & Post alignment


Downsamples to 100 Hz, time-point by time-point classification

In [1]:
%matplotlib inline

import os
from copy import deepcopy
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt

from mne import read_epochs
from mne import set_log_level

from scipy.stats import zscore

from hypertools.tools.align import align

# Classification stuff
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [2]:
set_log_level('ERROR')

## Settings

Note: Decode both to the stimulus, and to the response.

In [3]:
# Set data size
#   Note: these are set for arbitrary test data - need updating for 'real' data
n_epochs = 40
n_chs = 128
#n_times = 1001
n_per_cond = int(n_epochs / 2)

In [4]:
def maxabs(dat, dim):
    return np.max(np.abs(dat), dim)

In [5]:
# Globals

# Set the collection of ways to average across features
AVGS = {
    'maxabs' : maxabs,
    'max' : np.max, 
    'min' : np.min, 
    'mean' : np.mean, 
    'median' : np.median
}

# Classification Settings
K_FOLD = 3
AVG_TO_USE = 'mean'

# Initialize SVM classification object
#CLF = svm.SVC(kernel='linear')
CLF = svm.LinearSVC()

## Helper Functions

In [6]:
def extract_data(dat):
    """Organize data from MNE object, to data matrices and labels to be used for classification. 
    
    Parameters
    ----------
    dat : mne.Epochs object
        A subject's worth of epoched data.
    
    Returns
    -------
    labels : 1d array
        Labels for each trial type. 
    data : 3d array
        Epoched data matrix. 
    """

    # Check event codes there are, and unpack
    ev_counts = Counter(dat.events[:, 2])
    evc_a, evc_b = dat.event_id.keys()    
    n_evc_a, n_evc_b = ev_counts.values()
        
    # Generate labels
    lab_a = np.ones(shape=[n_per_cond]) * -1
    lab_b = np.ones(shape=[n_per_cond])

    # Extract trial data
    eps_a = dat[evc_a]._data[0:n_per_cond, 0:128, :]
    eps_b = dat[evc_b]._data[0:n_per_cond, 0:128, :]

    # Check all our shapes and sizes are correct
    assert len(lab_a) == np.shape(eps_a)[0]
    assert len(lab_b) == np.shape(eps_b)[0]
    assert len(lab_a) == len(lab_b)
    assert np.shape(eps_a)[0] == np.shape(eps_b)[0]
    
    # Collect all labels and trial data together
    data = np.concatenate([eps_a, eps_b], 0)
    labels = np.hstack([lab_a, lab_b])
    
    return data, labels


def make_2d(dat, z_score=True):
    """Reorganize a 3D matrix into a continuous 2D matrix. 
    
    Parameters
    ----------
    dat : 3d
        Epoched data matrix, as [n_epochs, n_channels, n_times]
    
    Returns
    -------
    2d array
        Continuous data matrix of epochs concatendat in time, as [n_channels, n_times_tot]
            Note: where n_times_tot = n_times * n_epochs
    """
    
    dat = np.concatenate(dat, 1)
    
    if z_score:
        dat = zscore(dat, 0)
    
    return dat


def make_3d(dat):
    """Reorganize a 2D matrix into the 3D trial structure matrix.
    
    Parameters
    ----------
    dat : 2d array
        Continuous data matrix of epochs concatendat in time, as [n_channels, n_times_tot]
            Note: where n_times_tot = n_times * n_epochs
        
    Results
    -------
    3d array
        Epoched data matrix, as [n_epochs, n_channels, n_times]
    """
    
    return np.stack(np.split(dat, n_epochs, 1))
    
    
def within_subj_classification(all_data, all_labels):
    """Run within subject classification within each subject for a list of subjects data.  
    
    Parameters
    ----------
    data : list of 3d array
        xx
    labels : list of 1d array
        xx
    
    Returns
    -------
    scores : 1d array
        xx
    """
    
    # Run cross-validated classification within each subject
    within_scores = np.zeros(shape=[len(all_data), K_FOLD])
    for s_ind, subj_data, subj_labels in zip(range(n_subjs), all_data, all_labels):
        within_scores[s_ind, :] = cross_val_score(CLF, feature_dat(subj_data), subj_labels, cv=K_FOLD)
    
    return within_scores


def btwn_subj_classication(all_data, all_labels):
    """Run classification between subjects.
    
    Parameters
    ----------
    all_data : list of 3d array
        Data for each subject.
    all_labels : list of 1d array
        Labels for each subject.
    
    Returns
    -------
    scores : list of float
        The classifications scores for each held out subject, as predicted from the group. 
    """

    scores = [None] * len(all_data)
    
    for ind, subj_data, subj_labels in zip(range(len(all_data)), all_data, all_labels):

        # Take a copy of the group data, and drop held out subject
        temp_data = deepcopy(all_data)
        temp_labels = deepcopy(all_labels)
        del temp_data[ind]
        del temp_labels[ind]

        # Collapse group for training the model
        group_data = feature_dat(np.concatenate(temp_data, 0))
        group_labels = np.concatenate(temp_labels, 0)

        # Train on group & classify left out subject
        CLF.fit(group_data, group_labels)
        scores[ind] = CLF.score(feature_dat(subj_data), subj_labels)

    return scores


def feature_dat(dat, avg_type=AVG_TO_USE):
    """Convert epochs 
    
    Parameters
    ----------
    dat : 3d array
        xx
    avg_type : {'max', 'min', 'mean', 'median', 'maxabs'}
        xx
    z_scores : bool
        xx
        
    Returns
    -------
    out : XX
        xx
    """

    avg = AVGS[avg_type]

    # Note: can add something here to select channels / time points
    out = avg(dat[:, :, :], 2)
    
    return out


def print_avg(label, score):
    print(label + ': {:1.2f}%'.format(score *100))


def print_avgs(label, scores):
    print(label + ':')
    for ind, score in enumerate(scores):
        print('\t{:1.0f} \t {:1.2f}'.format(ind, score))

## Data Organization / Loading

In [7]:
# Set data location for processed files
dat_path = '/Users/tom/Desktop/HyperEEG_Project/Data/proc/'

In [8]:
# Get list of available files
#  Note: this currently excludes first subject, because they are weird.
dat_files = [file for file in os.listdir(dat_path)[1:] if 'epo.fif' in file]

In [9]:
# Load all data
all_subjs = [read_epochs(os.path.join(dat_path, f_name),
                         preload=True, verbose=None) for f_name in dat_files]

# Downsample data
all_subjs = [dat.resample(100) for dat in all_subjs]

# Check how many subjects there are
n_subjs = len(all_subjs)

In [10]:
# # TESTS

# # Load single subject data - and fix channel subset
# dat = read_epochs(os.path.join(dat_path, dat_files[2]), preload=True, verbose=False)
# dat._data = dat._data[:, 0:128, :]

# # Make test list of multi-subj data
# n_group = 3
# all_subjs = [dat] * n_group
# n_subjs = len(all_subjs)

In [11]:
# Organize subject data into data and label matrices
all_data, all_labels = [], []

for subj in all_subjs:
    
    # Enforce a minimum number of trials - skip subj if not met
    if len(subj) < n_epochs:
        continue
        
    t_data, t_labels = extract_data(subj)
    all_data.append(t_data)
    all_labels.append(t_labels)

## Within Subject Classification (un-aligned)

Notes:
- Update to predict across windows of the trial

In [12]:
# Run within subject classification
within_scores = within_subj_classification(all_data, all_labels)

In [13]:
# Get average results - within and across subjects
within_subj_avgs = np.mean(within_scores, 1)
within_glob_avg = np.mean(within_subj_avgs)

In [14]:
# Check outcome - average across all subjects
print_avg('CV Within-Subj Prediction', within_glob_avg)

CV Within-Subj Prediction: 50.25%


In [15]:
# Check performance on each subject
print_avgs('Per Subj Within Predictions', within_subj_avgs)

Per Subj Within Predictions:
	0 	 0.50
	1 	 0.50
	2 	 0.40
	3 	 0.50
	4 	 0.50
	5 	 0.50
	6 	 0.50
	7 	 0.50
	8 	 0.63
	9 	 0.50
	10 	 0.50


## Between Subject Classification (un-aligned)

In [16]:
# Run prediction between subjects - on unaligned data
btwn_scores = btwn_subj_classication(all_data, all_labels)

In [17]:
# Get average results
avg_btwn_scores = np.mean(btwn_scores)

In [18]:
# Check outcome - average across all subjects
print_avg('Btwn-Subj Prediction', avg_btwn_scores)

Btwn-Subj Prediction: 52.50%


In [19]:
# Check performance on each subject
print_avgs('Btwn Subject Classification', btwn_scores)

Btwn Subject Classification:
	0 	 0.53
	1 	 0.50
	2 	 0.47
	3 	 0.50
	4 	 0.50
	5 	 0.62
	6 	 0.50
	7 	 0.50
	8 	 0.62
	9 	 0.53
	10 	 0.50


## Alignment (Hypertools)


In [20]:
# Data organization - extract matrices, and flatten to continuous data
all_data = [make_2d(dat) for dat in all_data]

In [21]:
# Do alignment
#  Note: this also switches orientation (takes the transpose) to match hypertools
aligned_data = align([dat.T for dat in all_data]) # Note: align assumes [n_samples x n_channels]
aligned_data = [dat.T for dat in aligned_data]
aligned_data = [make_3d(dat) for dat in aligned_data]

## Between Subject Classification (aligned)

In [22]:
# Check within subject prediction of aligned data
within_al1_scores = within_subj_classification(aligned_data, all_labels)
print_avg('Within Aligned', np.mean(within_al1_scores))
print_avgs('\nPer subj Within-Aligned', np.mean(within_al1_scores, 1))

Within Aligned: 51.59%

Per subj Within-Aligned:
	0 	 0.52
	1 	 0.53
	2 	 0.40
	3 	 0.53
	4 	 0.60
	5 	 0.49
	6 	 0.42
	7 	 0.48
	8 	 0.55
	9 	 0.57
	10 	 0.58


In [23]:
# Run prediction between subjects - on aligned data
btwn_al_scores = btwn_subj_classication(aligned_data, all_labels)

In [24]:
# Get average results
avg_btwn_al_scores = np.mean(btwn_al_scores)

In [25]:
# Check outcome - average across all subjects
print_avg('Btwn-Subj Prediction', avg_btwn_al_scores)

Btwn-Subj Prediction: 80.91%


In [26]:
# Check performance on each subject
print_avgs('Btwn Subject Classification', btwn_al_scores)

Btwn Subject Classification:
	0 	 0.88
	1 	 0.78
	2 	 0.65
	3 	 0.93
	4 	 0.85
	5 	 0.85
	6 	 0.90
	7 	 0.82
	8 	 0.70
	9 	 0.65
	10 	 0.90


#### CHECKS
Compare hyperaligned to unaligned data

In [27]:
# Note: these checks are for test cases which use a group of the same data copied over
#print(np.all(all_data[0] == all_data[1]))
#print(np.all(aligned_data[0] == aligned_data[1]))
#print(np.all(aligned_data[0] == all_data[0]))

## Check random rotations

In [28]:
# Random rotation matrix
#rot = np.random.random(size=n_chs*n_chs).reshape([n_chs, n_chs])

In [29]:
# Rotation by random matrix
#twod_dat = deepcopy(all_data)
#twod_dat = [np.dot(rot, dat) for dat in twod_dat]
#twod_dat_3d = [make_3d(dat) for dat in twod_dat]

In [30]:
# Check within subject prediction of random data
#within_rand_scores = within_subj_classification(twod_dat_3d, all_labels)
#print_avg('Within Rand', np.mean(within_rand_scores))

In [31]:
# Between subject classification
#rand_btwn_scores = btwn_subj_classication(twod_dat_3d, all_labels)

In [32]:
# Check outcome - average across all subjects
#avg_rand_btwn = np.mean(rand_btwn_scores)
#print_avg('Random Btwn-Subj Prediction', avg_rand_btwn)

## PyMVPA

Apply hyperalignment implementation from the PyMVPA package.

Note: this requires being in a Py2 environment with PyMVPA available.

In [33]:
from mvpa2.datasets.base import Dataset
from mvpa2.algorithms.hyperalignment import Hyperalignment

In [34]:
# Re-organize data into PyMVPA datasets objects
datasets = [Dataset(dat.T) for dat in all_data]

In [51]:
# Run hyperalignment, and get the transformation matrices
hyper_aligner = Hyperalignment(level2_niter=0, zscore_all=False, zscore_common=False)
hyper_aligner.train(datasets)
mappers = hyper_aligner(datasets)

In [52]:
# Apply the transformations to each dataset, and re-organize data
#   This applies the projection to the 2D data, transpose, and split back into epochs
aligned_datasets = []
for dataset, mapper in zip(datasets, mappers):
    aligned_datasets.append(make_3d(mapper.forward(dataset).samples.T))

In [53]:
# Check within subject prediction of aligned data
within_al2_scores = within_subj_classification(aligned_datasets, all_labels)
print_avg('Within Al2', np.mean(within_al2_scores))

Within Al2: 52.45%


In [54]:
# Between subject classification after PyMVPA hyperalignment
btwn_al2_scores = btwn_subj_classication(aligned_datasets, all_labels)

In [55]:
# Check average performance
avg_btwn_al2 = np.mean(btwn_al2_scores)
print_avg('Aligned-2 Btwn Scores', avg_btwn_al2)

Aligned-2 Btwn Scores: 85.00%


In [56]:
# 
print_avgs('Between Subj Aligned Data', btwn_al2_scores)

Between Subj Aligned Data:
	0 	 0.90
	1 	 0.85
	2 	 0.80
	3 	 0.90
	4 	 0.82
	5 	 0.85
	6 	 0.97
	7 	 0.93
	8 	 0.68
	9 	 0.68
	10 	 0.97


In [41]:
# Check if the rotation matrices are the same
#np.all(mappers[0].proj == mappers[1].proj)

#### Compare between the two hyperalignment implementations

In [42]:
# Check the magnitude of differences between aligned data
#diff = aligned_datasets[0] - aligned_data[0]

In [43]:
#print('Avg Magnitude Diff', np.mean(np.abs(diff)))
#print('Avg Magnitude Data', np.mean(np.abs(aligned_datasets[0])))

In [44]:
# Check number of overlapping points
#from __future__ import division
#np.sum(np.isclose(aligned_datasets[0], aligned_data[0])) / aligned_data[0].size

## Victory Party.

Soon...